In [7]:
import numpy as np
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.preprocessing import LabelEncoder
from math import log
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [22]:
def load_data():
  df1 = pd.read_csv('./Data/train1.csv', header = None)
  df2 = pd.read_csv('./Data/test1.csv', header = None)
  df = df1.append(df2)
  df.drop_duplicates(inplace = True)
  df.reset_index(inplace = True, drop = True)
  df.columns = ['peptide', 'aff', 'hla']
  df = df.groupby(['peptide','hla']).mean().reset_index()
  df = df.groupby('hla').filter(lambda x : len(x)>=20).reset_index(drop = True)
  
  df['tmp'] = df['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0)
  alpha = df.groupby('hla').agg({'tmp':'sum'})<4
  alpha = alpha.reset_index()
  df = pd.merge(df, alpha, on = 'hla', how = 'left')
  df = df[df['tmp_y'] == False]
  df.drop(columns = ['tmp_x','tmp_y'], inplace = True)
  
  print('Load_data...done')
  return df
df = load_data()
data = df['peptide'].apply(lambda x: ' '.join(x).split())

Load_data...done


In [23]:
df.shape

(133268, 3)

In [20]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data)]
model = Doc2Vec(documents, vector_size=80, window=4, min_count=1, workers=12)
model.save('./Data/Doc2Vec')

In [17]:
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=12)

In [18]:
model.save('./Data/Doc2Vec')

In [64]:
#alpha = data.apply(lambda x : model.infer_vector(x))

data = alpha.apply(pd.Series)

In [66]:
data.join(df).join(data)da

ValueError: columns overlap but no suffix specified: Index([0, 1, 2, 3, 4], dtype='object')

In [28]:
def labelenc(df, enc):
    enc = LabelEncoder().fit(df.iloc[:,0])
    for c in df.columns:
        df[c] = enc.transform(df[c])
    
    return df

def tear(serie, n, col):
    clus_df = pd.DataFrame(serie.copy())
    for i in range(1,n+1):
        col_name = col + '_P' + str(i)
        clus_df[col_name] = clus_df[col].apply(lambda x : x[i-1])

    cols = [c for c in clus_df.columns if c not in [col]]

    return clus_df[cols]

def hla_preprocess(df):
    df['allele_type'] = df['hla'].apply(
    lambda x: (x.startswith('DRB') and x[0:3]) or (x.startswith('HLA-DQ') and x[0:6]) or (x.startswith('HLA-DP') and x[0:6]) or x[0:3])

    hla_encoder = LabelEncoder()
    #hla_encoder.fit(df['hla'])
    #np.save('Data/hla_encoder_classes.npy', hla_encoder.classes_)
    hla_encoder.classes_ = np.load('./Data/hla_encoder_classes.npy')
    df['hla'] = hla_encoder.transform(df['hla'])

    allele_type_encoder = LabelEncoder()
    #allele_type_encoder.fit(df['allele_type'])
    #np.save('Data/allele_type_encoder_classes.npy', allele_type_encoder.classes_)
    allele_type_encoder.classes_ = np.load('./Data/allele_type_encoder_classes.npy')
    df['allele_type'] = allele_type_encoder.transform(df['allele_type'])

    return df
df = hla_preprocess(df)

In [24]:
data = alpha.apply(pd.Series)

In [47]:
#data = data.join(df[['hla', 'allele_type']])
y = df['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0)
xtrain, xvalid, ytrain, yvalid = train_test_split(data, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)
result = {}
# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = MinMaxScaler()
scl.fit(xtrain)
xtrain_svd_scl = scl.transform(xtrain)
xvalid_svd_scl = scl.transform(xvalid)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype float32, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5913175837323615

In [48]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=10000, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)

mlp.fit(xtrain_svd_scl, ytrain)
predictions = mlp.predict_proba(xvalid_svd_scl)[:,1]
result['mlp'] = roc_auc_score(yvalid,predictions)
result['mlp']

Iteration 1, loss = 0.66946146
Iteration 2, loss = 0.66845978
Iteration 3, loss = 0.66798997
Iteration 4, loss = 0.66776162
Iteration 5, loss = 0.66788004
Iteration 6, loss = 0.66771277
Iteration 7, loss = 0.66750397
Iteration 8, loss = 0.66733833
Iteration 9, loss = 0.66721302
Iteration 10, loss = 0.66716457
Iteration 11, loss = 0.66717385
Iteration 12, loss = 0.66716414
Iteration 13, loss = 0.66713614
Iteration 14, loss = 0.66703787
Iteration 15, loss = 0.66705089
Iteration 16, loss = 0.66682911
Iteration 17, loss = 0.66688483
Iteration 18, loss = 0.66682396
Iteration 19, loss = 0.66669830
Iteration 20, loss = 0.66675737
Iteration 21, loss = 0.66657302
Iteration 22, loss = 0.66681944
Iteration 23, loss = 0.66682368
Iteration 24, loss = 0.66687608
Iteration 25, loss = 0.66680707
Iteration 26, loss = 0.66657132
Iteration 27, loss = 0.66653353
Iteration 28, loss = 0.66665746
Iteration 29, loss = 0.66646652
Iteration 30, loss = 0.66665258
Iteration 31, loss = 0.66673072
Iteration 32, los

0.5986203232321383

In [50]:
rf = RandomForestClassifier(n_estimators = 1000, verbose=1, n_jobs = -1)
rf.fit(xtrain_svd_scl, ytrain)
predictions = rf.predict_proba(xvalid_svd_scl)[:,1]
result['RF'] = roc_auc_score(yvalid,predictions)
result['RF']

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   32.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   42.9s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.8s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    1.4s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    1.7s finished


0.6364097143032535

In [52]:
#rf = RandomForestClassifier(max_depth=3, n_estimators=n_estimator)
rf_enc = OneHotEncoder(categories='auto')
rf_lm = LogisticRegression(C=1.0)
rf_lm.fit(rf_enc.fit_transform(rf.apply(xtrain_svd_scl)), ytrain)

predictions = rf_lm.predict_proba(rf_enc.transform(rf.apply(xvalid_svd_scl)))[:,1]
result['RF_LR'] = roc_auc_score(yvalid,predictions)
result['RF_LR']

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    6.1s finished
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    1.5s f

0.6203815395543042